In [1]:
import librosa.display
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings(action="ignore")

import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile
import tensorflow as tf
from PIL import Image
from sklearn.utils import shuffle
from tqdm import tqdm
import numpy as np
import pandas as pd
import scipy.stats as st
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")
# warnings.filterwarnings(action='once')

import os
import shutil

# import geopandas
import librosa
import matplotlib.pyplot as plt

# plots
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

# keras
import tensorflow as tf
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    TimeSeriesSplit,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# import nltk
# from nltk.corpus import stopwords, wordnet
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer
# import torch
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import notebook, tqdm

%matplotlib inline

In [68]:
train_metadata= pd.read_csv("/app/_data/birdclef-2021/train_metadata.csv")
train_soundscape_labels = pd.read_csv(
    "/app/_data/birdclef-2021/train_soundscape_labels_full.csv"
)
train_metadata_orig= pd.read_csv("/app/_data/birdclef-2021/train_metadata_orig.csv")


In [69]:
train_metadata.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,date,filename,...,rating,time,url,date_1,geometry,file_path,frames,sr,duration,label_id
0,acafly,amegfi,"begging call, call, juvenile",35.3860,-84.1250,Empidonax virescens,Acadian Flycatcher,Mike Nelson,2012-08-12,XC109605.ogg,...,2.5,09:30,https://www.xeno-canto.org/109605,2012-08-12,POINT (-84.125 35.386),/app/_data/birdclef-2021/train_short_audio/aca...,2037586,32000,63.674563,0
1,acafly,NaN,call,9.1334,-79.6501,Empidonax virescens,Acadian Flycatcher,Allen T. Chartier,2000-12-26,XC11209.ogg,...,3.0,?,https://www.xeno-canto.org/11209,2000-12-26,POINT (-79.65009999999999 9.1334),/app/_data/birdclef-2021/train_short_audio/aca...,532933,32000,16.654156,0
2,acafly,NaN,call,5.7813,-75.7452,Empidonax virescens,Acadian Flycatcher,Sergio Chaparro-Herrera,2012-01-10,XC127032.ogg,...,3.0,15:20,https://www.xeno-canto.org/127032,2012-01-10,POINT (-75.7452 5.7813),/app/_data/birdclef-2021/train_short_audio/aca...,1508450,32000,47.139063,0
3,acafly,whwbec1,call,4.6717,-75.6283,Empidonax virescens,Acadian Flycatcher,Oscar Humberto Marin-Gomez,2009-06-19,XC129974.ogg,...,3.5,07:50,https://www.xeno-canto.org/129974,2009-06-19,POINT (-75.6283 4.6717),/app/_data/birdclef-2021/train_short_audio/aca...,450177,32000,14.068031,0
4,acafly,whwbec1,call,4.6717,-75.6283,Empidonax virescens,Acadian Flycatcher,Oscar Humberto Marin-Gomez,2009-06-19,XC129981.ogg,...,3.5,07:50,https://www.xeno-canto.org/129981,2009-06-19,POINT (-75.6283 4.6717),/app/_data/birdclef-2021/train_short_audio/aca...,1301142,32000,40.660688,0


In [70]:
train_metadata = train_metadata[['primary_label', 'secondary_labels', 'type', 'latitude', 'longitude',
       'date', 'filename',
       'rating', 'time', 'date_1', 'file_path',
       'frames', 'duration', 'label_id']]

In [71]:
train_metadata['month'] = train_metadata['date'].apply(lambda x: x.split('-')[1])
train_metadata['month'] = train_metadata['month'].astype('int')
train_metadata['year'] = train_metadata['date'].apply(lambda x: x.split('-')[0])
train_metadata['year'] = train_metadata['year'].astype('int')

In [67]:
train_metadata[['longitude', 'latitude']].describe()

,longitude,latitude
count,62874.000000,62874.000000
mean,-79.960828,24.472805
std,38.370726,22.108481
min,-165.781200,-53.162000
25%,-108.976100,6.246000
50%,-81.553400,31.741400
75%,-73.133333,41.198700
max,179.360700,78.928100


In [72]:
train_soundscape_labels

,row_id,site,audio_id,seconds,birds,date,file_path,longitude,latitude
0,7019_COR_5,COR,7019,5,nocall,2019-09-04,/app/_data/birdclef-2021/train_soundscapes/701...,-84.51,10.12
1,7019_COR_10,COR,7019,10,nocall,2019-09-04,/app/_data/birdclef-2021/train_soundscapes/701...,-84.51,10.12
2,7019_COR_15,COR,7019,15,nocall,2019-09-04,/app/_data/birdclef-2021/train_soundscapes/701...,-84.51,10.12
3,7019_COR_20,COR,7019,20,nocall,2019-09-04,/app/_data/birdclef-2021/train_soundscapes/701...,-84.51,10.12
4,7019_COR_25,COR,7019,25,nocall,2019-09-04,/app/_data/birdclef-2021/train_soundscapes/701...,-84.51,10.12
...,...,...,...,...,...,...,...,...,...
2395,54955_SSW_580,SSW,54955,580,nocall,2017-06-17,/app/_data/birdclef-2021/train_soundscapes/549...,-76.45,42.47
2396,54955_SSW_585,SSW,54955,585,grycat,2017-06-17,/app/_data/birdclef-2021/train_soundscapes/549...,-76.45,42.47
2397,54955_SSW_590,SSW,54955,590,grycat,2017-06-17,/app/_data/birdclef-2021/train_soundscapes/549...,-76.45,42.47
2398,54955_SSW_595,SSW,54955,595,nocall,2017-06-17,/app/_data/birdclef-2021/train_soundscapes/549...,-76.45,42.47


In [62]:
import librosa
import numpy as np
import torch
from torchaudio.transforms import MelSpectrogram

from PIL import Image


class MSG_Provider:
    def __init__(
        self,
        n_fft,
        n_mels,
        hop_length,
        sample_rate,
        target_msg_mels,
        target_msg_time_steps,
        device="cpu",
    ):

        self._msg_transform = MelSpectrogram(
            power=2.0,
            center=True,
            norm="slaney",
            onesided=True,
            win_length=None,
            pad_mode="reflect",
            n_fft=n_fft,
            n_mels=n_mels,
            sample_rate=sample_rate,
            hop_length=hop_length,
        ).to(device)

        self._device = device
        self._target_msg_mels = target_msg_mels
        self._target_msg_time_steps = target_msg_time_steps

    def msg(self, wave):
        wave = torch.tensor(wave.reshape([1, -1]).astype(np.float32)).to(self._device)

        msg = self._msg_transform(wave)[0].cpu().numpy()
        msg = librosa.power_to_db(msg)

        # if melspectrogram size mismatches with target, resize it
        if msg.shape != (self._target_msg_mels, self._target_msg_time_steps):
            msg = Image.fromarray(msg)
            msg = msg.resize(
                (self._target_msg_time_steps, self._target_msg_mels),
                Image.BICUBIC,
            )
            msg = np.array(msg)

        return msg.astype(np.float16)
